In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import pickle
import os
import seaborn as sns
import copy

from sklearn.metrics import normalized_mutual_info_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from vae import VAE
from loss_function import loss_function

In [3]:
# file located at '/data' in parent directory
path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'creditcard.csv')) 
print(path)
df = pd.read_csv(path)

/Users/HyeonWoo/Library/Mobile Documents/com~apple~CloudDocs/1. Career/KUBIG/data/creditcard.csv


In [4]:
display(df.head())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [6]:
df['Amount'] = df['Amount']/df['Amount'].max()

In [7]:
normal_data = df[df['Class']==0]
normal_data = normal_data.reset_index(drop=True)
display(normal_data.head())
print(normal_data.shape)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.005824,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.000105,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.014739,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.004807,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.002724,0


(284315, 31)


In [ ]:
novel_data = df[df['Class']==1]
novel_data = novel_data.reset_index(drop=True)
display(novel_data.head())
print(novel_data.shape)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.000000,1
1,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,0.020591,1
2,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,0.009339,1
3,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,0.002297,1
4,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,0.000039,1


(492, 31)


In [ ]:
train_size = int(normal_data.shape[0]*0.6) # 60% train
valid_size = int(normal_data.shape[0]*0.2) # 20% valid
test_size = int(normal_data.shape[0]*0.2) # 20% test

In [ ]:
print(train_size)
print(valid_size)
print(test_size)

170589
56863
56863


In [ ]:
train_data = normal_data[:train_size]
valid_data = normal_data[train_size:train_size+valid_size]
test_data = normal_data[train_size+valid_size:]

In [ ]:
display(train_data.head())
print(train_data.shape)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.005824,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.000105,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.014739,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.004807,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.002724,0


(170589, 31)


In [ ]:
test_data = pd.concat([test_data, novel_data])

In [ ]:
display(test_data.head())

display(test_data.tail())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
227452,145261.0,-0.627572,-0.086263,-0.089316,-2.385443,-0.370981,0.429405,0.093737,0.607601,-0.462058,...,-0.111377,-0.626828,0.416874,-0.467798,-0.712583,-0.721972,0.263590,0.210214,0.005126,0
227453,145261.0,-1.057854,-1.401509,-0.871779,-0.174520,3.930464,-0.124370,-0.165115,0.201862,-0.574865,...,0.411698,0.881594,0.255282,-0.903481,-0.954947,0.783469,-0.040398,0.017649,0.000251,0
227454,145262.0,-0.142559,0.668841,-1.665891,-0.868149,-0.716024,-0.089225,0.623346,-4.000170,-1.304595,...,2.755340,-1.001670,-0.859978,0.020269,0.308382,0.492557,0.618205,0.224843,0.012851,0
227455,145262.0,1.699428,-1.076039,-2.646925,-0.740128,2.062119,3.521782,-0.398007,0.799255,0.922402,...,-0.065158,-0.433283,0.105064,0.724002,-0.090990,-0.085987,-0.037582,-0.032766,0.006827,0
227456,145266.0,-0.382087,1.252484,-1.194991,-1.165406,0.912670,-1.344884,1.218899,0.119388,-0.710399,...,0.312633,0.916304,-0.239527,-0.299995,-0.294564,0.092928,0.262149,0.189222,0.000150,0


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
487,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,0.015180,1
488,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.000030,1
489,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,0.003032,1
490,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,0.009536,1
491,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,0.001655,1


In [ ]:
device = torch.device('cpu')

In [ ]:
zdims=2
hdims=15
batch_size=128
epochs = 30
input_dim = 29


In [ ]:
model = VAE(input_dim, zdims, hdims)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
class CCFDataset(Dataset) : 
    def __init__(self, df, transform=transforms.ToTensor()) :
        self.df = df.reset_index(drop=True)
        
    def __len__(self) :
        return self.df.shape[0]
    
    def __getitem__(self, idx) :
        # x, y
        # 코딩 못하면 이렇게 한줄에 코드 길어짐
        # ㅠㅠ
        x = torch.from_numpy(self.df.drop(['Class', 'Time'], axis=1).iloc[idx].values).type('torch.FloatTensor')
        y = self.df['Class'].iloc[idx]
        return x, y 

In [ ]:
train_dataset = CCFDataset(train_data)
valid_dataset = CCFDataset(valid_data)
test_dataset = CCFDataset(test_data)

In [ ]:
x, y = train_dataset[0]

In [ ]:
x

tensor([-1.3598, -0.0728,  2.5363,  1.3782, -0.3383,  0.4624,  0.2396,  0.0987,
         0.3638,  0.0908, -0.5516, -0.6178, -0.9914, -0.3112,  1.4682, -0.4704,
         0.2080,  0.0258,  0.4040,  0.2514, -0.0183,  0.2778, -0.1105,  0.0669,
         0.1285, -0.1891,  0.1336, -0.0211,  0.0058])

In [ ]:
x.shape

torch.Size([29])

In [ ]:
y

0

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def train(epoch) : 
    
    train_loss = 0
    loss_train = []
    loss_valid = []
    
    # get loss for whole validation set
    loss_valid.append(validate(1))
    
    model.train() # toggle train mode
    
    # get batch loss for train set and backpropate
    for batch_idx, (data, _) in enumerate(train_loader) :
        #data = Variable(data)
        optimizer.zero_grad()
        
        # Propagate
        recon_batch, mu, logvar = model(data)
        
        # Get loss value
        loss = loss_function(recon_batch, data, mu, logvar, batch_size, input_dim)
        
        # Backpropagate
        loss.backward()
        train_loss += loss.item()
        
        # Update weights
        optimizer.step()
            
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
      
    # append to list 'loss_train' instance to plot later
    loss_train.append(train_loss / len(train_loader.dataset))
    
    return loss_train, loss_valid

    
def validate(epoch) : 
    model.eval() # toggle inference mode
    valid_loss = 0
    
    
    for i, (data, _) in enumerate(test_loader) :
        #data = Variable(data)
        
        # Propagate
        with torch.no_grad() :
            recon_batch, mu, logvar = model(data)
        
        # Get loss value
        valid_loss += loss_function(recon_batch, data, mu, logvar, batch_size, input_dim).item()

    valid_loss /= len(test_loader.dataset)
    print('====> Validation set loss: {:.4f}'.format(valid_loss))
    
    return valid_loss

In [ ]:
loss_train = []
loss_valid = []
lowest_valid_loss = 0
best_model = copy.deepcopy(model)

for epoch in range(1, epochs+1) :
    temp_train, temp_valid = train(epoch)
    
    if epoch == 1 :
        lowest_valid_loss = temp_valid
    else :
        if temp_valid <= lowest_valid_loss :
            lowest_valid_loss = temp_valid
            best_model = copy.deepcopy(model)
            
    loss_train.append(temp_train)
    loss_valid.append(temp_valid)

====> Validation set loss: 0.0059
====> Epoch: 1 Average loss: -0.0154
====> Validation set loss: -0.0183
====> Epoch: 2 Average loss: -0.0352
====> Validation set loss: -0.0256
